# Setting & Import

In [1]:
import sys
import os

# 현재 파일(main.ipynb)이 있는 디렉토리의 부모 디렉토리를 sys.path에 추가
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
# INFO 
from config import *

# MAIN
from state.ver2 import *
from network.resnet import *
from trainer.ver1 import *
from eval.ver1 import *
from selfplay.ver1 import *
from tester.ver2 import *

# UTILS
from utils.saveLoad import *
from utils.valid_tool import *
from utils.setDevice import *
from utils.timer import *

In [3]:
State = select_state(STATE_DIM)
Selfplay = get_selfplay_class()

## Check

In [4]:
print(f"Using device: {device}")

Using device: mps


## Train

In [5]:
# 시각화용 
win_rates = []

# model 정의
model = Network(N_RESIDUAL_BLOCK, N_KERNEL, STATE_DIM, N_ACTIONS)
model = model.to(device)

self_play = Selfplay(model, TRAIN_TEMPERATURE, TRAIN_FREQUENCY, N_PLAYOUT)
train = TrainNetwork(model, BATCH_SIZE, LEARNING_RATE, LEARN_DECAY, learn_decay_index)
eval_network = EvalNetwork(model, EVAL_SELFPLAY, EVAL_TEMPERATURE, N_PLAYOUT)

In [6]:
start = time.time()
previous_time = start

# 경로 생성
make_directory(F_PATH)

# 학습
for i in range(total_iter):
    print(f'\n [ {i+1} ] --------------------------------')

    # selfplay 
    self_play(i)

    print_duration(previous_time, '>>> selfplay에 소요된 시간은')
    previous_time = time.time() 
    
    # 학습 
    if len(self_play.history) >= BATCH_SIZE:
        train(self_play.history)

        print_duration(previous_time, '>>> 학습에 소요된 시간은')
        previous_time = time.time() 

        save_model(train.model, f_name='latest_model_weight')

        if (i+1) % eval_index == 0:    
            eval_network(train.model)
            win_rates.append(eval_network.win_rate)

            print_duration(previous_time, '>>> eval에 소요된 시간은')
            previous_time = time.time() 

    if (i+1) % visualization_index == 0:
        eval_network.visualize_game(download=True, idx=i+1)

    if eval_network.updated:
        self_play.update_model(train.model)
        eval_network.updated = False

print_duration(start, '>>> 총 학습에 소요된 시간은')

'/Users/ijimin/Documents/GitHub/AiGO/Omok/Jimin/model/Omok_4' already exists.

 [ 1 ] --------------------------------
self play :  1 / 2000 | n_steps : 42
>>> selfplay에 소요된 시간은 0시간 1분 58초 입니다. 


 [ 2 ] --------------------------------


KeyboardInterrupt: 

In [ ]:
save_as_txt('config', config)

In [ ]:
visualize_win_rate(win_rates, path=F_PATH, download=True)
visualize_loss(train.losses, path=F_PATH, download=True)
visualize_n_steps(self_play.n_steps, window_size=100, path=F_PATH, download=True)